In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [3]:
!pip install transforms3d

In [4]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.integrate import cumtrapz
from transforms3d.axangles import axangle2mat
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

path = './'

train=pd.read_csv(path + 'train_features.csv')
# train_labels=pd.read_csv(path + 'train_labels.csv')
test=pd.read_csv(path + 'test_features.csv')
submission=pd.read_csv(path + 'sample_submission.csv')

In [5]:
# 데이터 분할
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [6]:
# Scaling
scaler=StandardScaler()
train[act_list]=scaler.fit_transform(train[act_list])
test[act_list]=scaler.transform(test[act_list])
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.367432,-0.845648,0.503707,-0.461266,-1.413128,-1.092639
1874996,3124,596,-0.326315,-0.845833,0.578875,-0.474109,-1.382884,-1.097377
1874997,3124,597,-0.300794,-0.859728,0.652722,-0.436694,-1.341672,-1.136169
1874998,3124,598,-0.253120,-0.886873,0.673002,-0.367051,-1.346630,-1.148831


In [7]:
# acceleromete, gyroscope sensor feature split
def sensor_split(data):
  X_acc = []
  X_gy = []

  for i in tqdm(data['id'].unique()):
      temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
      temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
      X_acc.append(temp_acc)
      X_gy.append(temp_gy)
      
  X_acc = np.array(X_acc).reshape(-1,600,3)
  X_gy = np.array(X_gy).reshape(-1,600,3)

  return X_acc, X_gy

In [8]:
X_train_acc, X_train_gy = sensor_split(train)

X_test_acc, X_test_gy = sensor_split(test)

100%|██████████| 782/782 [00:03<00:00, 235.34it/s]


In [9]:
# 데이터 증강
# Time Warping
sigma = 0.2
knot = 4

def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
    x_range = np.arange(X.shape[0])
    cs_x = CubicSpline(xx[:,0], yy[:,0])
    cs_y = CubicSpline(xx[:,1], yy[:,1])
    cs_z = CubicSpline(xx[:,2], yy[:,2])
    return np.array([cs_x(x_range),cs_y(x_range),cs_z(x_range)]).transpose()

def DistortTimesteps(X, sigma):
    tt = GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
    tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = [(X.shape[0]-1)/tt_cum[-1,0],(X.shape[0]-1)/tt_cum[-1,1],(X.shape[0]-1)/tt_cum[-1,2]]
    tt_cum[:,0] = tt_cum[:,0]*t_scale[0]
    tt_cum[:,1] = tt_cum[:,1]*t_scale[1]
    tt_cum[:,2] = tt_cum[:,2]*t_scale[2]
    return tt_cum

def TimeWarp(X, sigma):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])
    X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
    X_new[:,1] = np.interp(x_range, tt_new[:,1], X[:,1])
    X_new[:,2] = np.interp(x_range, tt_new[:,2], X[:,2])
    return X_new

# Permutation
def Permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(minSegLength, X.shape[0]-minSegLength, nPerm-1))
        segs[-1] = X.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]]:segs[idx[ii]+1],:]
        X_new[pp:pp+len(x_temp),:] = x_temp
        pp += len(x_temp)
    return (X_new)

# Rotation
def Rotation(X):
    axis = np.random.uniform(low=-1, high=1, size=X.shape[1])
    angle = np.random.uniform(low=-np.pi, high=np.pi)
    return np.matmul(X , axangle2mat(axis,angle))

# Jittering
def Jitter(X, sigma=0.05):
    myNoise = np.random.normal(loc=0, scale=sigma, size=X.shape)
    return X+myNoise

# Magnitude Warping
def MagWarp(X, sigma):
    return X * GenerateRandomCurves(X, sigma)

In [10]:
def ts_aug(data, method,sigma):
  new_data=[]
  for i in range(data.shape[0]):
    temp=list(method(data[i], sigma))
    new_data.append(temp)
  return np.array(new_data)

In [11]:
X_train_acc_comb = Permutation(Jitter(X_train_acc))
X_train_acc_perm = Permutation(X_train_acc)
X_train_acc_timew = ts_aug(X_train_acc, TimeWarp, 0.2)
X_train_acc_magw = ts_aug(X_train_acc, MagWarp, 0.2)

X_train_gy_comb = Permutation(Jitter(X_train_gy))
X_train_gy_perm = Permutation(X_train_gy)
X_train_gy_timew = ts_aug(X_train_gy, TimeWarp, 0.2)
X_train_gy_magw = ts_aug(X_train_gy, MagWarp, 0.2)

In [12]:
X_train_acc = np.append(X_train_acc, X_train_acc_comb, axis=0)
X_train_acc = np.append(X_train_acc, X_train_acc_perm, axis=0)
X_train_acc = np.append(X_train_acc, X_train_acc_timew, axis=0)
X_train_acc = np.append(X_train_acc, X_train_acc_magw, axis=0)

X_train_gy = np.append(X_train_gy, X_train_gy_comb, axis=0)
X_train_gy = np.append(X_train_gy, X_train_gy_perm, axis=0)
X_train_gy = np.append(X_train_gy, X_train_gy_timew, axis=0)
X_train_gy = np.append(X_train_gy, X_train_gy_magw, axis=0)

In [14]:
acc = [e for sl in X_train_acc for e in sl]

df_report = np.stack(acc, axis = 0)
df_acc = pd.DataFrame(df_report, columns= ['acc_x', 'acc_y', 'acc_z'])

df_acc

,acc_x,acc_y,acc_z
0,2.307314,-0.029939,-0.432104
1,2.421086,-0.063321,-0.504058
2,2.444664,-0.056749,-0.654199
3,2.428612,-0.116782,-0.573407
4,2.439475,-0.044220,-0.588886
...,...,...,...
9374995,-0.351245,-0.956397,0.421334
9374996,-0.310230,-0.957443,0.484434
9374997,-0.284353,-0.974000,0.546484
9374998,-0.237893,-1.005586,0.563721


In [15]:
# gy

gy = [e for sl in X_train_gy for e in sl]

df_report = np.stack(gy, axis = 0)
df_gy = pd.DataFrame(df_report, columns= ['gy_x', 'gy_y', 'gy_z'])

df_gy

,gy_x,gy_y,gy_z
0,0.011776,-0.410859,-0.461007
1,0.026484,-0.528052,-0.365459
2,-0.037962,-0.596041,-0.366767
3,0.066099,-0.725311,-0.401232
4,0.091969,-0.784094,-0.408413
...,...,...,...
9374995,-0.641622,-1.481999,-1.114402
9374996,-0.668820,-1.456255,-1.125095
9374997,-0.624772,-1.418792,-1.171078
9374998,-0.532588,-1.430131,-1.190554


In [16]:
# acc, gy 데이터프레임 병합

df_aug_result = pd.concat([df_acc, df_gy], axis = 1)
df_aug_result.insert(0, 'id', 0)
df_aug_result.insert(1, 'time', 1)
df_aug_result

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,1,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,1,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,1,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,1,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
9374995,0,1,-0.351245,-0.956397,0.421334,-0.641622,-1.481999,-1.114402
9374996,0,1,-0.310230,-0.957443,0.484434,-0.668820,-1.456255,-1.125095
9374997,0,1,-0.284353,-0.974000,0.546484,-0.624772,-1.418792,-1.171078
9374998,0,1,-0.237893,-1.005586,0.563721,-0.532588,-1.430131,-1.190554


In [17]:
df_aug_result.loc[600*0:600*0+600, 'time']

0      1
1      1
2      1
3      1
4      1
      ..
596    1
597    1
598    1
599    1
600    1
Name: time, Length: 601, dtype: int64

In [ ]:
#0~599까지 숫자를 담은 리스트 생성(for문 실행 속도 줄이기 위해)
list600 = [n for n in range(600)]

for i in range(int(len(df_aug_result)/600)):
  df_aug_result.loc[600*i:600*i+599, 'id'] = i              # id값 설정
  df_aug_result.loc[600*i:600*i+599, 'time'] = list600      # 리스트값 time열에 붙여넣기
  print(i, end=' ')

df_aug_result

##여기까지 np->df 변환하는 부분

In [19]:
train = df_aug_result
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
9374995,15624,595,-0.351245,-0.956397,0.421334,-0.641622,-1.481999,-1.114402
9374996,15624,596,-0.310230,-0.957443,0.484434,-0.668820,-1.456255,-1.125095
9374997,15624,597,-0.284353,-0.974000,0.546484,-0.624772,-1.418792,-1.171078
9374998,15624,598,-0.237893,-1.005586,0.563721,-0.532588,-1.430131,-1.190554


In [21]:
train_labels=pd.read_csv(path + 'train_labels.csv')
n = 4
train_labels_copy = train_labels.copy()
for i in range(0, n):
    train_labels = train_labels.append(train_labels_copy, ignore_index=True)
train_labels = train_labels.reset_index()
train_labels

,index,id,label,label_desc
0,0,0,37,Shoulder Press (dumbbell)
1,1,1,26,Non-Exercise
2,2,2,3,Biceps Curl (band)
3,3,3,26,Non-Exercise
4,4,4,26,Non-Exercise
...,...,...,...,...
15620,15620,3120,26,Non-Exercise
15621,15621,3121,26,Non-Exercise
15622,15622,3122,15,Dynamic Stretch (at your own pace)
15623,15623,3123,26,Non-Exercise


In [22]:
# 가속도 feature 추가
train['acc_t']  = train.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] ** 2 )**(1/3), axis=1)
test['acc_t']  = test.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 +  x['acc_z'] ** 2 )**(1/3), axis=1)

display(train)
display(test)

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t
0,0,0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007,1.766383
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459,1.829129
2,0,2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767,1.857368
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232,1.841091
4,0,4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413,1.846893
...,...,...,...,...,...,...,...,...,...
9374995,15624,595,-0.351245,-0.956397,0.421334,-0.641622,-1.481999,-1.114402,1.067241
9374996,15624,596,-0.310230,-0.957443,0.484434,-0.668820,-1.456255,-1.125095,1.076532
9374997,15624,597,-0.284353,-0.974000,0.546484,-0.624772,-1.418792,-1.171078,1.099222
9374998,15624,598,-0.237893,-1.005586,0.563721,-0.532588,-1.430131,-1.190554,1.114834


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t
0,3125,0,-0.249727,0.002786,0.202699,0.837933,1.212439,0.179961,0.469449
1,3125,1,-0.018931,0.296752,-0.231628,0.957733,1.318163,0.217914,0.521798
2,3125,2,0.119178,0.119214,-0.393070,0.507385,1.286118,0.176158,0.567658
3,3125,3,0.135533,-0.132967,-0.526513,0.410870,1.214796,0.177867,0.679157
4,3125,4,0.189646,0.064763,-0.792275,0.357015,1.170034,0.184028,0.874101
...,...,...,...,...,...,...,...,...,...
469195,3906,595,0.771160,-1.061284,1.235599,-0.150699,0.198219,-2.707576,1.480901
469196,3906,596,0.769913,-1.016963,1.185170,-0.395387,-0.110361,-2.566486,1.447297
469197,3906,597,0.804762,-1.000674,1.122728,-0.437428,-0.310409,-2.314917,1.427600
469198,3906,598,0.771076,-0.904187,1.095665,-0.436282,-0.408929,-1.977082,1.377285


In [23]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [24]:
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 7])
X.shape

TensorShape([15625, 600, 7])

In [25]:
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(15625, 61)

In [26]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(600,7)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(X,y, epochs=1000, batch_size=2048, validation_split=0.2)

Epoch 1/1000
7/7 [==============================] - 2s 210ms/step - loss: 1.9494 - accuracy: 0.5522 - val_loss: 1.5442 - val_accuracy: 0.6022
Epoch 2/1000
7/7 [==============================] - 1s 188ms/step - loss: 1.9428 - accuracy: 0.5544 - val_loss: 1.5506 - val_accuracy: 0.5990
Epoch 3/1000
7/7 [==============================] - 1s 189ms/step - loss: 1.9407 - accuracy: 0.5535 - val_loss: 1.5260 - val_accuracy: 0.6086
Epoch 4/1000
7/7 [==============================] - 1s 187ms/step - loss: 1.9289 - accuracy: 0.5584 - val_loss: 1.5905 - val_accuracy: 0.5926
Epoch 5/1000
7/7 [==============================] - 1s 188ms/step - loss: 1.9590 - accuracy: 0.5546 - val_loss: 1.5381 - val_accuracy: 0.6077
Epoch 6/1000
7/7 [==============================] - 1s 187ms/step - loss: 1.9380 - accuracy: 0.5549 - val_loss: 1.5170 - val_accuracy: 0.6115
Epoch 7/1000
7/7 [==============================] - 1s 189ms/step - loss: 1.9470 - accuracy: 0.5533 - val_loss: 1.5409 - val_accuracy: 0.6064
Epoch 

In [36]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 7])
test_X.shape

TensorShape([782, 600, 7])

In [37]:
prediction=model.predict(test_X)
prediction

array([[4.02601319e-04, 2.73025804e-03, 8.89635658e-07, ...,
        5.36541529e-02, 1.20884339e-04, 1.23251812e-03],
       [1.42636462e-04, 7.38309836e-03, 1.48385482e-08, ...,
        5.55037445e-07, 7.90467220e-06, 4.31393459e-03],
       [9.18840989e-04, 2.06125187e-05, 1.10993045e-04, ...,
        1.01113571e-07, 5.25134710e-05, 3.99232935e-03],
       ...,
       [1.29690452e-04, 1.95393382e-04, 2.34057597e-06, ...,
        1.00165687e-06, 2.38954145e-02, 8.49631266e-04],
       [3.74725460e-05, 7.28266255e-04, 5.83100264e-05, ...,
        9.45799111e-05, 3.79984244e-03, 7.67444400e-03],
       [5.15439735e-09, 1.88975682e-04, 1.40899112e-02, ...,
        9.28768059e-05, 4.42824501e-07, 3.53855453e-03]], dtype=float32)

In [38]:
prediction.shape

(782, 61)

In [39]:
submission.iloc[:,1:]=prediction

In [40]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,4.026013e-04,2.730258e-03,8.896357e-07,8.236593e-07,0.081648,4.744366e-05,2.606433e-03,1.050270e-03,0.000878,0.005795,1.529884e-01,4.694952e-02,0.000150,1.394533e-01,1.738111e-01,1.503786e-05,4.382837e-03,1.420230e-08,0.000862,1.254043e-04,1.368561e-04,1.776964e-04,9.596048e-03,0.010927,6.735633e-02,2.290378e-05,0.105226,5.776024e-04,0.004442,1.964375e-04,0.068490,1.332804e-02,3.296605e-04,1.238788e-02,1.922570e-04,0.000013,0.001162,0.007127,1.918744e-05,0.001828,0.000004,6.058272e-05,6.046506e-06,0.001077,7.509820e-04,0.002669,1.963704e-04,2.707099e-07,0.000105,1.578672e-02,0.002688,0.000690,5.645788e-05,8.672014e-04,1.710538e-03,7.713864e-04,0.000092,0.000029,5.365415e-02,1.208843e-04,0.001233
1,3126,1.426365e-04,7.383098e-03,1.483855e-08,5.532043e-05,0.000272,1.466201e-05,2.976896e-04,2.939127e-05,0.001344,0.001975,6.973025e-07,5.628613e-04,0.000023,2.685286e-07,5.660162e-04,1.938312e-03,6.813048e-05,1.884069e-08,0.000338,5.382509e-08,2.148025e-04,7.923564e-04,5.678396e-03,0.000197,1.781887e-04,9.560642e-06,0.930175,1.687729e-04,0.001753,2.327016e-03,0.000360,9.661999e-06,1.704291e-04,3.080565e-06,7.533584e-04,0.000040,0.013694,0.001022,3.695557e-06,0.000008,0.000003,1.868070e-05,4.181909e-04,0.001725,1.773788e-03,0.000006,2.581200e-06,6.918873e-03,0.004435,2.606284e-04,0.002244,0.000047,2.655432e-05,2.807147e-05,4.784987e-03,1.503354e-05,0.000013,0.000388,5.550374e-07,7.904672e-06,0.004314
2,3127,9.188410e-04,2.061252e-05,1.109930e-04,5.949580e-05,0.000330,5.200006e-05,7.372148e-04,3.057852e-06,0.369204,0.001180,4.254920e-04,4.882451e-04,0.000005,4.973615e-11,7.140119e-06,5.096288e-05,5.553681e-06,1.977208e-01,0.000028,2.675237e-10,4.300875e-07,4.545587e-06,1.472024e-08,0.000007,9.987921e-07,1.901606e-03,0.071740,9.323321e-04,0.001930,6.916779e-04,0.000047,3.391553e-05,7.197266e-04,2.621800e-04,7.190486e-08,0.000004,0.001208,0.000242,3.635544e-06,0.000017,0.002237,1.753756e-07,3.650570e-08,0.000996,2.755552e-05,0.239274,9.149231e-05,2.134792e-04,0.069490,1.032628e-03,0.001201,0.000017,1.414306e-05,3.770529e-07,2.886818e-02,7.790506e-04,0.000077,0.000541,1.011136e-07,5.251347e-05,0.003992
3,3128,2.959197e-04,1.448399e-03,7.593156e-04,5.299076e-05,0.000113,1.898575e-04,2.077300e-03,8.753050e-04,0.005951,0.001181,1.733538e-03,6.205294e-03,0.001774,1.159333e-04,7.490383e-03,1.264081e-03,7.803583e-05,1.510237e-04,0.001355,1.662094e-04,1.117725e-04,2.233462e-04,7.554395e-05,0.000239,4.490175e-03,1.875068e-06,0.795465,1.247003e-03,0.005720,1.182946e-02,0.001579,6.026138e-04,2.005690e-03,1.488495e-03,1.887043e-03,0.004295,0.000152,0.000794,2.376620e-03,0.000064,0.010443,1.126339e-04,1.935065e-04,0.001822,2.832449e-04,0.000247,5.334943e-06,4.614290e-03,0.010686,7.939833e-03,0.001910,0.002933,1.105305e-02,4.934691e-05,1.153783e-03,8.893959e-04,0.000671,0.056823,3.485422e-03,5.915339e-04,0.016166
4,3129,9.739913e-05,1.086821e-04,1.057370e-05,6.560873e-04,0.014959,1.330385e-03,4.305796e-04,4.094356e-03,0.060284,0.009711,3.324688e-06,1.809163e-03,0.000231,5.229228e-04,2.594919e-04,1.980091e-04,1.377767e-04,2.397862e-04,0.007601,2.067778e-03,1.801373e-05,7.103659e-05,9.161630e-02,0.000008,3.960274e-04,2.397254e-04,0.740540,5.314383e-04,0.007538,1.311752e-07,0.000912,2.050992e-04,5.335832e-03,2.768980e-03,1.757194e-03,0.000032,0.005992,0.002747,2.381622e-04,0.000303,0.000537,1.576522e-06,5.121663e-04,0.001000,1.195220e-04,0.001689,5.661090e-05,1.213186e-04,0.002046,6.543439e-04,0.008680,0.005008,9.531563e-05,8.015912e-06,1.298268e-03,8.134100e-03,0.000173,0.003053,3.701970e-05,2.696534e-04,0.000503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,1.

In [42]:
submission.to_csv('baseline_aug_1.csv', index=False)